In [13]:
import pyarrow.parquet as pq
import pandas as pd
df = pd.read_parquet('../vkrecsys/train_interactions.parquet')

In [ ]:
#### разделяем данные на тест и валид

In [21]:
# Определяем размеры частей для каждого нового датафрейма
# Первую часть вообще пока не использую - не хватает мощности на обработку 
n_rows = len(df)
part_1_size = int(n_rows * 0.6)
part_2_size = int(n_rows * 0.2) + int(n_rows * 0.6)
part_3_size = n_rows - int(n_rows * 0.2) - int(n_rows * 0.6)

# Разделяем датафрейм на три части
df_part_1 = df.iloc[:part_1_size]
df_part_test = df.iloc[part_1_size:part_2_size]
df_part_valid = df.iloc[-part_3_size:]

print("Размер первой части:", df_part_test.shape)
#print("Размер второй части:", df_part_test.shape)
print("Размер третьей части:", df_part_valid.shape)

Размер первой части: (43700184, 7)
Размер третьей части: (43700186, 7)


In [22]:
df_part_test.head()

,user_id,item_id,timespent,like,dislike,share,bookmarks
58266912,30182,268571,30,0,0,0,0
58266913,26720,9745,228,1,0,0,0
58266914,181334,256078,1,0,0,0,0
58266915,175024,49430,21,0,0,0,0
58266916,132194,147488,4,0,0,0,0


In [23]:
df_part_valid.head()

,user_id,item_id,timespent,like,dislike,share,bookmarks
101967096,144614,191362,55,0,0,0,0
101967097,156080,17204,122,0,0,0,0
101967098,75032,320340,37,0,0,0,0
101967099,147401,168077,53,0,0,0,0
101967100,15329,316419,115,0,0,0,0


In [ ]:
#### объединяем с данными об айтемах

In [24]:
cosim = pd.read_csv('../vkrecsys/_class_similar_items.csv')

In [25]:
test_pairs = pd.merge(df_part_test, cosim, on=["item_id"])

In [26]:
test_pairs.head()

,user_id,item_id,timespent,like,dislike,share,bookmarks,source_id,duration,class
0,30182,268571,30,0,0,0,0,2679,58,157636
1,26720,9745,228,1,0,0,0,13472,16,5609
2,181334,256078,1,0,0,0,0,2900,50,150136
3,175024,49430,21,0,0,0,0,10641,22,28815
4,132194,147488,4,0,0,0,0,7154,26,86444


In [27]:
valid_pairs = pd.merge(df_part_valid, cosim, on=["item_id"])

In [28]:
valid_pairs.head()

,user_id,item_id,timespent,like,dislike,share,bookmarks,source_id,duration,class
0,144614,191362,55,0,0,0,0,7637,90,111734
1,156080,17204,122,0,0,0,0,13753,119,9772
2,75032,320340,37,0,0,0,0,13200,39,188456
3,147401,168077,53,0,0,0,0,555,52,98356
4,15329,316419,115,0,0,0,0,12952,54,186165


In [ ]:
##### объединяем с данными о юзерах

In [29]:
users = pd.read_parquet('../vkrecsys/users_meta.parquet.parquet')

In [32]:
test_pairs_users = pd.merge(test_pairs, users, on=["user_id"])

In [34]:
valid_pairs_users = pd.merge(valid_pairs, users, on=["user_id"])

In [33]:
test_pairs_users.head()

,user_id,item_id,timespent,like,dislike,share,bookmarks,source_id,duration,class,gender,age
0,30182,268571,30,0,0,0,0,2679,58,157636,1,31
1,26720,9745,228,1,0,0,0,13472,16,5609,2,43
2,181334,256078,1,0,0,0,0,2900,50,150136,1,34
3,175024,49430,21,0,0,0,0,10641,22,28815,1,22
4,132194,147488,4,0,0,0,0,7154,26,86444,2,41


In [35]:
valid_pairs_users.head()

,user_id,item_id,timespent,like,dislike,share,bookmarks,source_id,duration,class,gender,age
0,144614,191362,55,0,0,0,0,7637,90,111734,2,20
1,156080,17204,122,0,0,0,0,13753,119,9772,1,18
2,75032,320340,37,0,0,0,0,13200,39,188456,2,23
3,147401,168077,53,0,0,0,0,555,52,98356,2,39
4,15329,316419,115,0,0,0,0,12952,54,186165,2,38


In [36]:
valid_pairs_users.drop(columns=['timespent'], inplace=True)
test_pairs_users.drop(columns=['timespent'], inplace=True)

In [37]:
valid_pairs_users.head()

,user_id,item_id,like,dislike,share,bookmarks,source_id,duration,class,gender,age
0,144614,191362,0,0,0,0,7637,90,111734,2,20
1,156080,17204,0,0,0,0,13753,119,9772,1,18
2,75032,320340,0,0,0,0,13200,39,188456,2,23
3,147401,168077,0,0,0,0,555,52,98356,2,39
4,15329,316419,0,0,0,0,12952,54,186165,2,38


In [38]:
valid_pairs_users.to_parquet('../vkrecsys/fin_valid.parquet', index=False)

In [39]:
test_pairs_users.to_parquet('../vkrecsys/fin_test.parquet', index=False)

In [ ]:
#### прогоняю через катбуст

In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# Загрузим объединённый датафрейм
#merged_df = pd.read_parquet('../vkrecsys/df_train.parquet')
merged_df = pd.read_parquet('../vkrecsys/fin_test.parquet')


## добавила сортировку
merged_df = merged_df.query("(like + dislike + share + bookmarks) >= 1")

# Определим целевые и предикторные переменные
#X = merged_df[['user_id', 'item_id', 'dislike', 'share', 'bookmarks', 'gender', 'age', 'percent_watch']]
X = merged_df[['user_id', 'item_id', 'gender', 'age']]
y = merged_df['like']
#'source_id', 'duration', 'class', 
# Разделим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Преобразование категориальных признаков в числовой формат
#cat_features_indices = [0, 1, 2]  # индексы категориальных признаков: user_id, item_id, gender
cat_features_indices = ['user_id','item_id','gender']
#'source_id', 'class', 
# Создаём объекты Pool для CatBoost
train_pool = Pool(X_train, y_train, cat_features=cat_features_indices)
test_pool = Pool(X_test, y_test, cat_features=cat_features_indices)

# Настраиваем параметры модели
model_params = {
    'iterations': 300,
    'depth': 8,
    'learning_rate': 0.001,
    'loss_function': 'Logloss',
    'eval_metric': 'AUC'
}

# Создаём и обучаем модель
model = CatBoostClassifier(**model_params)
model.fit(train_pool, eval_set=test_pool, verbose=False)

# Оценка качества модели на тестовых данных
predictions = model.predict(test_pool)
accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy:.4f}')
AUC = roc_auc_score(y_test, predictions)
print(f'Roc-Auc: {AUC:.4f}')

Accuracy: 0.9094
Roc-Auc: 0.7559


In [2]:
test_pairs = pd.read_csv('../vkrecsys/test_pairs.csv.csv')
users= pd.read_parquet('../vkrecsys/users_meta.parquet.parquet')
cosim = pd.read_csv('../vkrecsys/_class_similar_items.csv')
# Применим модель к тестовым данным
test_pairs_merg = test_pairs.merge(users, on=['user_id'], how='left')
test_pairs_merged = test_pairs_merg.merge(cosim, on=['item_id'], how='left')

In [3]:
predictions = model.predict_proba(test_pairs_merged[X.columns])[:, 1]
test_pairs_merged['predict'] = predictions
#удаляю колонки возраст пол
test_pairs_merged.drop(columns=['gender', 'age'], inplace=True)

In [4]:
# Сохранение результатов
test_pairs_merged.to_csv('../vkrecsys/real_cat_pred.csv', index=False)
#print("Предсказания сохранены в файл 'predictions.csv'")

In [ ]:
test_pairs = pd.read_parquet('../vkrecsys/fin_valid.parquet')
test_pairs.drop(columns=['like', 'dislike', 'share', 'bookmarks'], inplace=True)

In [ ]:
predictions = model.predict_proba(test_pairs[X.columns])[:, 1]
valid_df['predict'] = predictions
#удаляю колонки возраст пол
valid_df.drop(columns=['gender', 'age'], inplace=True)

In [ ]:
# Сохранение результатов
test_pairs_merged.to_csv('../vkrecsys/valid_cat_pred.csv', index=False)
#print("Предсказания сохранены в файл 'predictions.csv'")

In [ ]:
model.save_model("../vkrecsys/catboost_model.cbm")

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

# Загружаем данные из файлов
merged_data = pd.read_parquet('../vkrecsys/fin_test.parquet')

In [2]:
# Создаем целевую переменную для обучения
target_column = 'like'
merged_data[target_column] = merged_data['like'].astype(int)

# Выбираем нужные столбцы для обучения
features_columns = ['gender', 'age', 'source_id', 'duration', 'class']
X = merged_data[features_columns]
y = merged_data[target_column]

# Масштабируем данные
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Преобразуем данные в формат, подходящий для LSTM
X_train = X_scaled.reshape((len(X), 1, len(features_columns)))

# Создание и компиляция модели
model = Sequential()
model.add(LSTM(units=64, input_shape=(1, len(features_columns)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['auc'])


# Обучение модели
history = model.fit(X_train, y, epochs=6, batch_size=32, validation_split=0.2, verbose=1)
model.save('../vkrecsys/RNN_fin.keras')
model.save('../vkrecsys/RNN_fin.h5')

C:\Users\Margarita\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/6
1092505/1092505 ━━━━━━━━━━━━━━━━━━━━ 3677s 3ms/step - auc: 0.6951 - loss: 0.1737 - val_auc: 0.7029 - val_loss: 0.1765
Epoch 2/6
1092505/1092505 ━━━━━━━━━━━━━━━━━━━━ 3902s 4ms/step - auc: 0.7004 - loss: 0.1729 - val_auc: 0.7045 - val_loss: 0.1763
Epoch 3/6
1092505/1092505 ━━━━━━━━━━━━━━━━━━━━ 3896s 4ms/step - auc: 0.7015 - loss: 0.1727 - val_auc: 0.7053 - val_loss: 0.1762
Epoch 4/6
1092505/1092505 ━━━━━━━━━━━━━━━━━━━━ 3883s 4ms/step - auc: 0.7022 - loss: 0.1728 - val_auc: 0.7054 - val_loss: 0.1764
Epoch 5/6
1092505/1092505 ━━━━━━━━━━━━━━━━━━━━ 3873s 4ms/step - auc: 0.7021 - loss: 0.1726 - val_auc: 0.7034 - val_loss: 0.1764
Epoch 6/6
1092505/1092505 ━━━━━━━━━━━━━━━━━━━━ 3888s 4ms/step - auc: 0.7017 - loss: 0.1727 - val_auc: 0.7043 - val_loss: 0.1765


In [15]:
test_pairs = pd.read_parquet('../vkrecsys/fin_valid.parquet')

In [16]:
test_pairs.drop(columns=['like', 'dislike', 'share', 'bookmarks'], inplace=True)

In [ ]:
test_features = test_pairs[features_columns]
test_X = scaler.transform(test_features)
test_X = test_X.reshape((len(test_X), 1, len(features_columns)))

# Делаем предсказания
predictions = model.predict(test_X)

# Добавляем предсказанные значения в DataFrame
test_pairs['predict'] = predictions

# Сохраняем результаты в новый CSV-файл
test_pairs.to_csv('../vkrecsys/valid_test_rnn.csv', index=False)

In [8]:
test_pairs = pd.read_csv('../vkrecsys/test_pairs.csv.csv')
users= pd.read_parquet('../vkrecsys/users_meta.parquet.parquet')
cosim = pd.read_csv('../vkrecsys/_class_similar_items.csv')
# Применим модель к тестовым данным
test_pairs_merg = test_pairs.merge(users, on=['user_id'], how='left')
test_pairs_merged = test_pairs_merg.merge(cosim, on=['item_id'], how='left')



In [9]:
test_pairs_merged.head()

,user_id,item_id,gender,age,source_id,duration,class
0,1,7363,2,35,5119,9,4121
1,1,73770,2,35,2720,5,43298
2,1,75700,2,35,11601,32,44365
3,1,81204,2,35,4296,10,47630
4,1,110249,2,35,18812,38,64601


In [10]:
test_features = test_pairs_merged[features_columns]
test_X = scaler.transform(test_features)
test_X = test_X.reshape((len(test_X), 1, len(features_columns)))

# Делаем предсказания
predictions = model.predict(test_X)

# Добавляем предсказанные значения в DataFrame
test_pairs['predict'] = predictions

# Сохраняем результаты в новый CSV-файл
test_pairs.to_csv('../vkrecsys/real_test_rnn.csv', index=False)

51723/51723 ━━━━━━━━━━━━━━━━━━━━ 73s 1ms/step


In [13]:
test_pairs.head()

,user_id,item_id,source_id,duration,class,gender,age
0,144614,191362,7637,90,111734,2,20
1,156080,17204,13753,119,9772,1,18
2,75032,320340,13200,39,188456,2,23
3,147401,168077,555,52,98356,2,39
4,15329,316419,12952,54,186165,2,38
